In [4]:
import sys, os
sys.path.insert(0, os.path.abspath("../"))
from n3.parse import parse_n3

In [5]:
data1 = parse_n3("""@prefix : <http://example.org/> .
{
  :Socrates a :Human .
  :Plato :thinks true .
  :Plato :thinks false .
} a :Statement .""").data
data1

{ <http://example.org/Socrates> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.org/Human> .
<http://example.org/Plato> <http://example.org/thinks> True .
<http://example.org/Plato> <http://example.org/thinks> False } <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.org/Statement> .

In [6]:
data2 = parse_n3("""@prefix : <http://example.org/> .
{
    ?S :thinks ?t .
    ?S :thinks ?f .
    ?P a :Human .
} a :Statement .""").data
data2

{ ?S <http://example.org/thinks> ?t .
?S <http://example.org/thinks> ?f .
?P <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.org/Human> } <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.org/Statement> .

In [16]:
def check_binding(var_node, other, bindings):
    if var_node.var_id in bindings:
        return bindings[var_node.var_id] == other.idx_val()
    bindings[var_node.var_id] = other.idx_val()
    return True

def match_terms(term1, term2, bindings):
    if term1.is_concrete():
        if term2.is_concrete():
            ret = term1 == term2
        else:
            ret = check_binding(term2, term1, bindings)
    else:
        if term2.is_concrete():
            ret = check_binding(term1, term2, bindings)
        else: # doesnt' have to be same variable
            ret = True

    # print(term1, term2, ret)
    return ret
    
def match_tp(tp1, tp2, bindings):
    return match_terms(tp1.s, tp2.s, bindings) and match_terms(tp1.p, tp2.p, bindings) and match_terms(tp1.o, tp2.o, bindings)

In [13]:
def match(graph1, idx1, graph2, bindings):
    if idx1 == len(graph1):
        print(bindings)
        return
    
    tp1 = graph1.triple_at(idx1)
    
    for tp2 in graph2.triples():
        bindings2 = bindings.copy()
        # print(tp1, "<>", tp2, match_tp(tp1, tp2, bindings2))
        if match_tp(tp1, tp2, bindings2):
            match(graph1, idx1 + 1, graph2, bindings2)

In [17]:
graph1 = data1.triple_at(0).s.model
graph2 = data2.triple_at(0).s.model

match(graph1, 0, graph2, {})

{'P': 'http://example.org/Socrates', 'S': 'http://example.org/Plato', 't': True, 'f': False}
{'P': 'http://example.org/Socrates', 'S': 'http://example.org/Plato', 'f': True, 't': False}
